In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime as dt
import os

def getNow():
    now       = dt.now().strftime('%Y-%m-%d--%H-%M-%S')
    nowFolder = os.path.join('./logFolder', now)
    return nowFolder

In [2]:
tf.reset_default_graph()

In [3]:
N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 4 + 0.1*(np.random.rand(N)-0.5)
y = y.reshape((-1, 1))
tMask = np.random.choice([True, False], (N,), p=[0.8, 0.2])

In [4]:
with tf.variable_scope('trainData'):
    inpTrain = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inpTrain')
    outTrain = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='outTrain')

with tf.variable_scope('testData'):
    inpTest = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inpTest')
    outTest = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='outTest')

with tf.variable_scope('model'):
    W = tf.Variable(initial_value=np.random.rand(2, 1), name='W', dtype=tf.float32)
    b = tf.Variable(initial_value=0, name='b', dtype=tf.float32)

    with tf.variable_scope('train'):
        yHatTrain = tf.matmul(inpTrain,  W )+ b
        errTrain  = tf.reduce_mean((outTrain - yHatTrain)**2, name='errTrain')
        
    with tf.variable_scope('test'):
        yHatTest = tf.matmul(inpTest,  W )+ b
        errTest  = tf.reduce_mean((outTest - yHatTest)**2, name='errTest')

with tf.variable_scope('optimizer'):
    lr = tf.Variable(0.01, dtype=tf.float32, name='learningRate')
    opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(errTrain)

init = tf.global_variables_initializer()

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
folder = getNow()

with tf.Session() as sess:
    writer = tf.summary.FileWriter(folder, sess.graph)
    sess.run(init)
    
    for i in range(5001):
        sess.run( opt, feed_dict={
            inpTrain : X[tMask, :],
            outTrain : y[tMask, :],
            lr       : 0.01
             })
        
        if (i % 100) == 0:
            errTrainVal, errTestVal = sess.run( [errTrain, errTest], feed_dict={
                inpTrain : X[tMask, :],
                outTrain : y[tMask, :],
                inpTest : X[~tMask, :],
                outTest : y[~tMask, :]})
            print('Training: {}, Testing: {}'.format(errTrainVal, errTestVal))
            
    
    writer.close()

print('Final Values')
print('-'*30)
print('Training: {}, Testing: {}'.format(errTrainVal, errTestVal))
    
print('tensorboard --logdir={}'.format( folder ))

Training: 40.29877471923828, Testing: 40.64850997924805
Training: 19.58721351623535, Testing: 19.752025604248047
Training: 8.304112434387207, Testing: 8.367670059204102
Training: 3.0473554134368896, Testing: 3.063396692276001
Training: 0.9970974922180176, Testing: 0.9944819211959839
Training: 0.33822399377822876, Testing: 0.32972249388694763
Training: 0.16195061802864075, Testing: 0.152193084359169
Training: 0.11852984130382538, Testing: 0.10893143713474274
Training: 0.10413378477096558, Testing: 0.09508880972862244
Training: 0.09480762481689453, Testing: 0.08642720431089401
Training: 0.08628194034099579, Testing: 0.07859578728675842
Training: 0.07802779227495193, Testing: 0.0710354745388031
Training: 0.07006215304136276, Testing: 0.06374838948249817
Training: 0.062452010810375214, Testing: 0.05679236724972725
Training: 0.05525629594922066, Testing: 0.05021993815898895
Training: 0.04852062091231346, Testing: 0.044071998447179794
Training: 0.042277880012989044, Testing: 0.03837799653410